# Exploring sequence of Tn10 

In [3]:
import wgregseq

First we read the FASTA file obtained from Genebank.

In [4]:
with open ("tn10.fasta", "r") as file:
    data = file.read().split('\n')[1:]
    sequence = "".join(data)

Organization of tetR/tetA regulation:
- two operators that can be bound independently by TetR
- tetA is repressed by both tetO1 and tetO2
- tetR is repressed only by tetO1
- Affinity of tetO2 to TetR is about twice as high as tetO1

![](tn10_tet.png)

From Genebank, we can find the positions for *tetA* and *tetR*. The repressor gene is reversed, so we will have to obtain the complementary sequence in case we are interested in the actual sequences.

In [5]:
# Exact positions from Genebank
tetR_pos = [4702, 5328]
tetA_pos = [5407, 6612]

tetA = sequence[tetA_pos[0]-1:tetA_pos[1]]
tetR = wgregseq.complement_seq(sequence[tetA_pos[0]-1:tetA_pos[1]])

In [6]:
# Positions taken from Bertram 2008
tetO2_pos = [tetA_pos[0] - 28, tetA_pos[0] - 10]
tetO1_pos = [tetA_pos[0] - 58, tetA_pos[0] - 40]

tetO2 = sequence[tetO2_pos[0]:tetO2_pos[1]+1]
tetO1 = sequence[tetO1_pos[0]:tetO1_pos[1]+1]

In [7]:
tetO2

'TCCCTATCAGTGATAGAGA'

In [8]:
tetO1

'ACTCTATCATTGATAGAGT'

In [9]:
rev_tetO1 = wgregseq.complement_seq(tetO1, rev=True)
rev_tetO1

'ACTCTATCAATGATAGAGT'

In [10]:
rev_tetO2 = wgregseq.complement_seq(tetO2, rev=True)
rev_tetO2

'TCTCTATCACTGATAGGGA'

In [11]:
# TSS estimated from -10 region
tetA_TSS = tetO2_pos[0]+1
tetR_TSS1 = tetO2_pos[0] - 19
tetR_TSS2 = tetO1_pos[0] - 8

In [12]:
P_tetA = sequence[tetA_TSS-36:tetA_TSS+1]
P_tetA

'TTGACACTCTATCATTGATAGAGTTATTTTACCACTC'

In [13]:
P_tetR1 = wgregseq.complement_seq(sequence[tetO1_pos[1]-6:tetA_pos[0] - 8], rev=True)
P_tetR1

'TTCTCTATCACTGATAGGGAGTGGTAAAATAACTCTAT'

In [14]:
P_tetR2 = wgregseq.complement_seq(sequence[tetO1_pos[0]-8:tetO2_pos[0]-1], rev=True)
P_tetR2

'TGGTAAAATAACTCTATCAATGATAGAGTGTCAACAA'

In [15]:
len(P_tetA)

37

In [16]:
lavUV5 = 'TCGAGTTTACACTTTATGCTTCCGGCTCGTATAATGTGTGG'

## Constructs

### RegSeq tetA

In [18]:
regseq_tetA = sequence[tetA_TSS-115:tetA_TSS+45]
regseq_tetA

'GACCTCATTAAGCAGCTCTAATGCGCTGTTAATCACTTTACTTTTATCTAATCTAGACATCATTAATTCCTAATTTTTGTTGACACTCTATCATTGATAGAGTTATTTTACCACTCCCTATCAGTGATAGAGAAAAGTGAAATGAATAGTTCGACAAAGA'

### RegSeq tetR

In [24]:
regseq_tetR1 = wgregseq.complement_seq(sequence[tetR_TSS1-45:tetR_TSS1+115])
regseq_tetR1

'TAGATCTGTAGTAATTAAGGATTAAAAACAACTGTGAGATAGTAACTATCTCAATAAAATGGTGAGGGATAGTCACTATCTCTTTTCACTTTACTTATCAAGCTGTTTCTAGCGTAACCATTAATGCAATGAGCTACGGTACCCCTAACCGGAATAGTAC'

In [25]:
regseq_tetR2 = wgregseq.complement_seq(sequence[tetR_TSS2-45:tetR_TSS2+115])
regseq_tetR2

'TAGTGAAATGAAAATAGATTAGATCTGTAGTAATTAAGGATTAAAAACAACTGTGAGATAGTAACTATCTCAATAAAATGGTGAGGGATAGTCACTATCTCTTTTCACTTTACTTATCAAGCTGTTTCTAGCGTAACCATTAATGCAATGAGCTACGGTA'

### LacUV5 + individual operators downstream

In [22]:
lacUV5_tetO1 = lavUV5 + tetO1
lacUV5_tetO1

'TCGAGTTTACACTTTATGCTTCCGGCTCGTATAATGTGTGGACTCTATCATTGATAGAGT'

In [23]:
lacUV5_tetO2 = lavUV5 + tetO2
lacUV5_tetO2

'TCGAGTTTACACTTTATGCTTCCGGCTCGTATAATGTGTGGTCCCTATCAGTGATAGAGA'